### Installs

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.7/823.7 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.0 MB/s eta 0:00:000m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 8.2 MB/s eta 0:00:00


In [3]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 76.0 MB/s eta 0:00:00m eta 0:00:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 94.0 MB/s eta 0:00:000m eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.5 MB/s eta 0:00:000m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.9 MB/s eta 0:00:00
  Using cached wrapt-1.14.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (81 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [4]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.6 MB/s eta 0:00:00


In [5]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 73.7 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pdf2image

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [143]:
!pip install -U langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.2/834.2 kB 15.9 MB/s eta 0:00:00m eta 0:00:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.169
    Uninstalling langchain-0.0.169:
      Successfully uninstalled langchain-0.0.169


### Imports 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from openai.error import InvalidRequestError
from langchain.prompts import PromptTemplate
import datetime
import pandas as pd

In [3]:
with open('secret', 'r') as fr:
    secret = fr.readline().strip()

### Manula exploration

#### Knowledgebase

In [4]:
loader = UnstructuredPDFLoader('./mdma_ib22.pdf')

In [5]:
data = loader.load()

In [6]:
len(data), len(data[0].page_content), data[0].page_content[300000:301800]

(1,
 729200,
 'ly greater for MDMA group compared to the placebo group (P=0.037), and placebo-subtracted Cohen’s d effect size was very large (d=1.4, CI: -0.074, 2.874). Change in LSAS scores from the Baseline Visit to 6- month follow-up showed similar positive results (P=0.036), with a Cohen’s d effect size of 1.1 (CI: -0.307, 2.527). The study safety and efficacy data were published [89].\n\n6.3.3 Anxiety Associated with Life-Threatening Illness\n\nMAPS is studying the effects of MDMA-assisted therapy on people experiencing anxiety as they face a potentially life-threatening illness. A manuscript containing results was published in Scientific Reports in November 2020. Participants with anxiety from life-threatening illnesses were randomized to receive MDMA (125 mg MDMA HCl) or placebo in combination with two eight-hour therapy sessions. At the primary endpoint, the MDMA group had a greater mean (SD) reduction in State-Trait Anxiety Inventory (STAI) Trait scores, -23.5 (SD:12.3), indi

In [7]:
chunk_size = 1800

In [8]:
len(data[0].page_content) / chunk_size

405.1111111111111

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=50, length_function=len)

In [10]:
chunks = text_splitter.split_text(data[0].page_content)

Created a chunk of size 6463, which is longer than the specified 1800
Created a chunk of size 6564, which is longer than the specified 1800
Created a chunk of size 2491, which is longer than the specified 1800
Created a chunk of size 1921, which is longer than the specified 1800
Created a chunk of size 2024, which is longer than the specified 1800
Created a chunk of size 1841, which is longer than the specified 1800
Created a chunk of size 2871, which is longer than the specified 1800
Created a chunk of size 3882, which is longer than the specified 1800
Created a chunk of size 3122, which is longer than the specified 1800
Created a chunk of size 2228, which is longer than the specified 1800
Created a chunk of size 2695, which is longer than the specified 1800
Created a chunk of size 2124, which is longer than the specified 1800
Created a chunk of size 1854, which is longer than the specified 1800
Created a chunk of size 1947, which is longer than the specified 1800
Created a chunk of s

In [11]:
len(chunks)

471

#### Embeddings

In [89]:
embeddings_gpt35 = OpenAIEmbeddings(openai_api_key=secret, model='gpt-3.5-turbo')

In [90]:
knowledge_base_gpt35 = FAISS.load_local(folder_path='./knowledge_base/', index_name='maps_gpt35', embeddings=embeddings_gpt35)
#knowledge_base_gpt35 = FAISS.from_texts(chunks, embeddings_gpt35)

In [91]:
#knowledge_base_gpt35.save_local(folder_path='./knowledge_base/', index_name='maps_gpt35')

In [92]:
embeddings_ada002 = OpenAIEmbeddings(openai_api_key=secret, model='text-embedding-ada-002')

In [93]:
knowledge_base_ada002 = FAISS.load_local(folder_path='./knowledge_base/', index_name='maps_ada002', embeddings=embeddings_ada002)
# knowledge_base = FAISS.from_texts(chunks, embeddings_ada002)

In [ ]:
# knowledge_base_ada002.save_local(folder_path='./knowledge_base/', index_name='maps_ada002')

#### Chatbots

In [17]:
llm_davinci003 = ChatOpenAI(openai_api_key=secret, model_name='text-davinci-003', max_tokens=1000, temperature=0.7)
llm_gpt35 = ChatOpenAI(openai_api_key=secret, model_name='gpt-3.5-turbo', max_tokens=1000, temperature=0.7)

#### Queries

##### Custom

In [188]:
prompt_template = """Use the following pieces of context and your internal knowledge base to answer the question at the end. Don't try to make up an answer if you don't know.

{context}
Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [189]:
knowledge_base = knowledge_base_ada002
embeddings = embeddings_ada002
llm = llm_gpt35
context = 5
chain_type = 'stuff' 
prompt = PROMPT

In [190]:
user_question = "Summarize the results for efficacy for primary endpoints across MAPS studies."
docs = knowledge_base.similarity_search(user_question, k=context)
chain = load_qa_chain(llm, chain_type=chain_type, prompt=prompt, verbose=True)

In [191]:
with get_openai_callback() as cb:
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=False)
response['output_text'], cb



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context and your internal knowledge base to answer the question at the end. Don't try to make up an answer if you don't know.

Positive efficacy data from Phase 3 study MAPP1 supports the conclusions of Phase 2 studies, showing that MDMA as an adjunct to supportive therapy is statistically superior for PTSD treatment compared to therapy paired with an inactive placebo. Responder analysis showed that 88% of participants in the MDMA-AT group had a clinically significant improvement, and 67% no longer met diagnostic criteria for PTSD.

MDMA-assisted therapy showed significantly greater reduction in PTSD symptoms compared to therapy with an inactive placebo group, with no increase in reported adverse events. Clinician-rated functional impairment also improved with MDMA therapy.

Tables summarizing ecstasy use history, effects of MDMA, clinical studies compar

('The Phase 3 study MAPP1 and Phase 2 studies showed that MDMA as an adjunct to supportive therapy is statistically superior for PTSD treatment compared to therapy paired with an inactive placebo. Responder analysis showed that 88% of participants in the MDMA-AT group had a clinically significant improvement, and 67% no longer met diagnostic criteria for PTSD. MDMA-assisted therapy showed significantly greater reduction in PTSD symptoms compared to therapy with an inactive placebo group, with no increase in reported adverse events. Clinician-rated functional impairment also improved with MDMA therapy. The CAPS-IV and updated CAPS-5 scores trended downward after two dosing sessions of MDMA-assisted therapy, and symptom severity remained reduced at least 12 months later.',
 Tokens Used: 536
 	Prompt Tokens: 388
 	Completion Tokens: 148
 Successful Requests: 1
 Total Cost (USD): $0.001072)

##### Stuffing 

In [108]:
knowledge_base = knowledge_base_ada002
embeddings = embeddings_ada002
llm = llm_gpt35
context = 10
chain_type = 'stuff' 

In [109]:
user_question = "Summarize the results for efficacy for primary endpoints across MAPS studies."
docs = knowledge_base.similarity_search(user_question, k=context)
chain = load_qa_chain(llm, chain_type=chain_type, verbose=True)

In [ ]:
while True:
    try:
        with get_openai_callback() as cb:
            response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=False)
        break
    except InvalidRequestError as err:
        print(f"Too long context: {len(docs)}")
        docs = docs[:-1]
    
response['output_text'], cb

##### Refine

In [59]:
knowledge_base = knowledge_base_ada002
embeddings = embeddings_ada002
llm = llm_gpt35
context = 9
chain_type = 'refine' 

In [60]:
user_question = "Summarize the results for efficacy for primary endpoints across MAPS studies."
docs = knowledge_base.similarity_search(user_question, k=context)
chain = load_qa_chain(llm, chain_type=chain_type, verbose=True)

In [ ]:
with get_openai_callback() as cb:
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=False)
response['output_text'], cb

##### Summary Summary

In [196]:
prompt_template = """Summarize the following context with as few tokens as possible.
{context}
Summary:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context"]
)

In [197]:
knowledge_base = knowledge_base_ada002
embeddings = embeddings_ada002
llm = llm_gpt35
context = 5
chain_type = 'stuff' 
prompt = PROMPT

In [198]:
user_question = "Summarize the results for efficacy for primary endpoints across MAPS studies."
docs = knowledge_base.similarity_search(user_question, k=context)
chain = load_qa_chain(llm, chain_type=chain_type, prompt=prompt, verbose=True)

In [199]:
for doc_idx in range(len(docs)):
    with get_openai_callback() as cb:
        response = chain({"input_documents": [docs[doc_idx]]}, return_only_outputs=False)
    text = response['output_text']
    docs[doc_idx].page_content = text



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Summarize the following context with as few tokens as possible.
Positive efficacy data from Phase 3 study MAPP1 supports the conclusions of Phase 2 studies, showing that MDMA as an adjunct to supportive therapy is statistically superior for PTSD treatment compared to therapy paired with an inactive placebo. Responder analysis showed that 88% of participants in the MDMA-AT group had a clinically significant improvement, and 67% no longer met diagnostic criteria for PTSD.
Summary:

> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Summarize the following context with as few tokens as possible.
MDMA-assisted therapy showed significantly greater reduction in PTSD symptoms compared to therapy with an inactive placebo group, with no increase in reported adverse events. Clinician-rated functional 

In [201]:
user_question = "Summarize the results for efficacy for primary endpoints across MAPS studies."
chain = load_qa_chain(llm, chain_type=chain_type, verbose=True)

In [202]:
with get_openai_callback() as cb:
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=False)
response['output_text'], cb



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
MDMA with supportive therapy is effective for treating PTSD, according to Phase 3 study MAPP1. 88% of participants showed significant improvement and 67% no longer met diagnostic criteria.

MDMA therapy reduces PTSD symptoms and improves functional impairment.

MDMA Investigator's Brochure includes tables on ecstasy use, effects, clinical studies, adverse reactions, and body changes during dosing.

MAPS conducted 8 blinded and 4 open-label studies on MDMA-assisted therapy for PTSD, highlighting Phase 2 studies MP-1, MP-8, MP-12, and MP-16 while summarizing smaller Phase 2 studies due to design differences.

MDMA-assisted therapy reduces PTSD symptoms long-term and future studies will use updated CAPS-

("I'm sorry, I cannot provide a summary of the results for efficacy for primary endpoints across MAPS studies as the context does not provide that information.",
 Tokens Used: 251
 	Prompt Tokens: 221
 	Completion Tokens: 30
 Successful Requests: 1
 Total Cost (USD): $0.0005020000000000001)

### Knowledge base preparation

In [4]:
def create_knowledge_base(document, embed_model_name='text-embedding-ada-002', chunk_size=1800, alias='maps'):
    loader = UnstructuredPDFLoader(document)
    embeddings = OpenAIEmbeddings(openai_api_key=secret, model=embed_model_name)
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=25, length_function=len)

    data = loader.load()
    assert len(data) == 1
    print(f'Document loaded, size: {len(data[0].page_content)}')
    
    chunks = text_splitter.split_text(data[0].page_content)
    print(f"Getting {len(chunks)} chunks of text")
    
    knowledge_base = FAISS.from_texts(chunks, embeddings)
    index_name = f"{embed_model_name}_{alias}_{chunk_size}"
    print(f"Creating {index_name} index with {embed_model_name} embeddings")
    knowledge_base.save_local(folder_path='./knowledge_base/', index_name=index_name)
    return knowledge_base, index_name

def load_knowledge_base(index_name):
    embed_model_name = index_name.split('_')[0]
    print(f"Loading {index_name} index, knowledge base will be queried using {embed_model_name} embeddings")
    embeddings = OpenAIEmbeddings(openai_api_key=secret, model=embed_model_name)
    knowledge_base = FAISS.load_local(folder_path='./knowledge_base/', index_name=index_name, embeddings=embeddings)
    return knowledge_base

In [207]:
maps_indexes = ['text-embedding-ada-002_maps_600', 'text-embedding-ada-002_maps_1800']

### Strategy of inference

In [5]:
def get_response(user_question, knowledge_base, result, context_size=7, temperature=0.7, strategy='stuff', chat_model_name='gpt-3.5-turbo', verbose=False):
    custom_stuff_template = """Use the following pieces of context and your internal knowledge base to answer the question at the end. Don't try to make up an answer if you don't know.\n\n{context}\nQuestion: {question}\nAnswer:"""
    custom_stuff_prompt = PromptTemplate(template=custom_stuff_template, input_variables=["context", "question"]) 
    
    summary_template = """Summarize the following context with as few tokens as possible.\n\n{context}\nSummary:"""
    summary_prompt = PromptTemplate(template=summary_template, input_variables=["context"])
    
    result['temperature'] = temperature
    result['context_size'] = context_size
    result['strategy'] = strategy
    result['user_question'] = user_question
    result['chat_model_name'] = chat_model_name
    
    result['completion_tokens'] = 0
    result['prompt_tokens'] = 0 
    result['total_tokens'] = 0 
    result['total_cost'] = 0
    result['answer'] = ""
    result['success'] = False
    result['real_context_size'] = 0
    result['total_time'] = 0
    start_time = datetime.datetime.now()
    
    llm = ChatOpenAI(openai_api_key=secret, model_name=chat_model_name, max_tokens=1000, temperature=temperature)
    docs = knowledge_base.similarity_search(user_question, k=context_size)
    if strategy == 'sum_sum':
        summary_chain = load_qa_chain(llm, chain_type='stuff', prompt=summary_prompt, verbose=verbose)
        for doc_idx in range(len(docs)):
            with get_openai_callback() as cb:
                response = summary_chain({"input_documents": [docs[doc_idx]]}, return_only_outputs=False)
            text = response['output_text']
            docs[doc_idx].page_content = text
            result['completion_tokens'] += cb.completion_tokens
            result['prompt_tokens'] += cb.prompt_tokens 
            result['total_tokens'] += cb.total_tokens 
            result['total_cost'] += cb.total_cost
        chain = load_qa_chain(llm, chain_type='stuff', verbose=verbose)
    elif strategy in ['stuff', 'refine']: 
        chain = load_qa_chain(llm, chain_type=strategy, verbose=verbose)
    else:
        chain = load_qa_chain(llm, chain_type='stuff', verbose=verbose, prompt=custom_stuff_prompt)

    while True:
        try:
            with get_openai_callback() as cb:
                response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=False)
            result['completion_tokens'] += cb.completion_tokens
            result['prompt_tokens'] += cb.prompt_tokens 
            result['total_tokens'] += cb.total_tokens 
            result['total_cost'] += cb.total_cost
            result['answer'] = response['output_text']
            result['success'] = True
            result['real_context_size'] = len(docs)
            result['total_time'] = (datetime.datetime.now() - start_time).total_seconds()
            return True
        except InvalidRequestError as err:
            docs = docs[:-1]
            if len(docs) == 0:
                result['success'] = False
                result['total_time'] = (datetime.datetime.now() - start_time).total_seconds()
                return False

In [6]:
result = {}
knowledge_base = load_knowledge_base('text-embedding-ada-002_maps_1800')
user_question = "Summarize the results for efficacy for primary endpoints across MAPS studies."
success = get_response(user_question, knowledge_base, result, context_size=16, temperature=0.7, strategy='sum_sum', verbose=True)
result

Loading text-embedding-ada-002_maps_1800 index, knowledge base will be queried using text-embedding-ada-002 embeddings


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Summarize the following context with as few tokens as possible.

Page 84 of 253

MAPS U.S.

Phase 3 Efficacy Data

MDMA Investigator’s Brochure 14th Ed: 18 March 2022

Efficacy data from the pivotal Phase 3 study, MAPP1, were positive and support the conclusions of the Phase 2 studies. MDMA as an adjunct to supportive therapy was statistically superior for PTSD treatment in CAPS-5 severity scores from Baseline to Primary Endpoint, 8 weeks after three blinded dosing sessions, in comparison to therapy paired with an inactive placebo (p<0.0001, d=0.91) [30]. The pooled mean Total Severity CAPS-5 scores for MAPP1 are shown in Figure 5, as reported in a comparison of MDMA-assisted therapy versus placebo with therapy groups at Baseline (T1); three weeks after dosing sess

{'temperature': 0.7,
 'context_size': 16,
 'strategy': 'sum_sum',
 'user_question': 'Summarize the results for efficacy for primary endpoints across MAPS studies.',
 'chat_model_name': 'gpt-3.5-turbo',
 'completion_tokens': 1289,
 'prompt_tokens': 8118,
 'total_tokens': 9407,
 'total_cost': 0.018814000000000004,
 'answer': 'The document mentions that multiple Phase 2 and Phase 3 studies were conducted on MDMA-assisted therapy for PTSD, with positive efficacy data from Phase 3 study MAPP1 supporting the conclusions of Phase 2 studies. The studies aimed to explore the reproducibility and persistence of treatment outcomes in chronic PTSD patients and those who failed to respond to previous therapies. In studies of MDMA-assisted therapy for PTSD, CAPS-IV scores decreased after treatment and remained lower at least 12 months later, with some participants no longer meeting PTSD criteria. The updated CAPS-5 was used in completed and planned studies. The primary endpoint for the study of effec

### Results

In [7]:
questions = [
    "What are all the possible physical and psychological effects of taking MDMA? Please only list effects that people would notice, instead of granular physiological changes.",
    "What are all the possible physical and psychological effects of taking MDMA?",
    "Are there any differences in how MDMA treats males vs. females? If so, how confident is the literature in these differences?",
    "Are there any differences in how MDMA treats males vs. females?",
    "What is the time course of taking MDMA? When do the effects start, when is the peak effect and how long does it last?",
    "When do the MDMA effects start, when is the peak effect and how long does it last?",
    "Are there differences in how MDMA and R-MDMA effect people? How confident are we in these differences?",
    "Are there differences in how MDMA and R-MDMA effect people?",
    "Summarize the results for efficacy for primary endpoints across MAPS studies."
]

In [8]:
knowledge_base_600 = load_knowledge_base('text-embedding-ada-002_maps_600')
knowledge_base_1800 = load_knowledge_base('text-embedding-ada-002_maps_1800')

Loading text-embedding-ada-002_maps_600 index, knowledge base will be queried using text-embedding-ada-002 embeddings
Loading text-embedding-ada-002_maps_1800 index, knowledge base will be queried using text-embedding-ada-002 embeddings


In [ ]:
results = []
results_df = None
verbose = True
for question in questions:
    for knowledge_base, chunk_size in [(knowledge_base_600, 600), (knowledge_base_1800, 1800)]:
            if chunk_size == 600:
                for strategy in ['stuff', 'custom_stuff']:
                    for context_size in [8, 24]:
                        for temperature in [0.2, 0.7]:
                            print("##\n##\n\n", strategy, chunk_size, context_size, temperature)
                            result = {'embedding_model_name': "text-embedding-ada-002", 'document': 'maps', 'run': 1, 'chunk_size': chunk_size}
                            success = get_response(question, knowledge_base, result, context_size=context_size, temperature=temperature, strategy=strategy, verbose=verbose)
                            results.append(result.copy())
                            results_df = pd.DataFrame(results)
                            results_df.to_csv('./results.csv')
            else: # 1800
                for strategy in ['stuff', 'custom_stuff']:
                    for context_size in [8]:
                        for temperature in [0.2, 0.7]:
                            print("##\n##\n\n", strategy, chunk_size, context_size, temperature)
                            result = {'embedding_model_name': "text-embedding-ada-002", 'document': 'maps', 'run': 1, 'chunk_size': chunk_size}
                            success = get_response(question, knowledge_base, result, context_size=context_size, temperature=temperature, strategy=strategy, verbose=verbose)
                            results.append(result.copy())
                            results_df = pd.DataFrame(results)
                            results_df.to_csv('./results.csv')
                for strategy in ['refine', 'sum_sum']:
                    for context_size in [24]:
                        for temperature in [0.2, 0.7]:
                            print("##\n##\n\n", strategy, chunk_size, context_size, temperature)
                            result = {'embedding_model_name': "text-embedding-ada-002", 'document': 'maps', 'run': 1, 'chunk_size': chunk_size}
                            success = get_response(question, knowledge_base, result, context_size=context_size, temperature=temperature, strategy=strategy, verbose=verbose)
                            results.append(result.copy())
                            results_df = pd.DataFrame(results)
                            results_df.to_csv('./results.csv')

##
##

 stuff 600 8 0.2


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Downing, J., The psychological and physiological effects of MDMA on normal volunteers. J Psychoactive Drugs, 1986. 18(4): p. 335-40.

Lester, S.J., et al., Cardiovascular effects of 3,4-methylenedioxymethamphetamine. A double-blind, placebo-controlled trial. Ann Intern Med, 2000. 133(12): p. 969-73.

Bedi, G., et al., Effects of MDMA on sociability and neural response to social threat and social reward. Psychopharmacology (Berl), 2009. 207(1): p. 73-83.

6.2.3 Physiological Effects

Physiological effects from published literature are reported in the sub-sections below; data derived from MAPS sponsored Phase 2 and Phase 3 studies are included in Section 6.4 Safety of MDMA in Hum

### Random Digging

In [81]:
responsement = knowledge_base.docstore._dict['a7ab7322-165c-4ae2-9f40-6233d8c94d7f']
#document

In [82]:
knowledge_base.index.ntotal

471

In [83]:
knowledge_base.index.reconstruct(1)

array([-0.00087333,  0.02169342,  0.00307516, ..., -0.03947533,
       -0.02155188, -0.02524465], dtype=float32)

In [84]:
idx = knowledge_base.index_to_docstore_id[1]
doc = knowledge_base.docstore.search(idx)
idx

'ca4365d1-4999-449f-847b-78002c9ac739'

### Backup

In [ ]:
!gsutil cp /home/adam_kolar_healthmode_com/code/mini/pdfbot/pdfbot.ipynb gs://hm-ml-experiments/chatGPT/pdfbot.ipynb